In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [49]:
data_df = pd.read_csv('C:\\Users\\nikhi\\Downloads\\RBC Hackathon\\Py\\load_weather_processed.csv')
data_df.tail()

,Year,Month,Day,Time (LST),Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Wind Spd Flag,Stn Press (kPa),Hmdx,Wind Chill,Load [MW]
10229,2020,12,31,19,3.3,0.4,81.0,27.0,15.0,NaN,101.49,NaN,NaN,1521.92
10230,2020,12,31,20,2.9,-0.3,79.0,27.0,16.0,NaN,101.58,NaN,NaN,1463.28
10231,2020,12,31,21,2.3,-0.6,81.0,27.0,12.0,NaN,101.65,NaN,NaN,1404.85
10232,2020,12,31,22,2.2,-0.8,81.0,27.0,13.0,NaN,101.70,NaN,NaN,1359.48
10233,2020,12,31,23,2.2,-1.3,77.0,27.0,14.0,NaN,101.76,NaN,NaN,1369.47


In [50]:
X, y = data_df.iloc[:,:-1],data_df.iloc[:,-1]

In [51]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10234 entries, 0 to 10233
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 10234 non-null  int64  
 1   Month                10234 non-null  int64  
 2   Day                  10234 non-null  int64  
 3   Time (LST)           10234 non-null  int64  
 4   Temp (°C)            10219 non-null  float64
 5   Dew Point Temp (°C)  10216 non-null  float64
 6   Rel Hum (%)          10216 non-null  float64
 7   Wind Dir (10s deg)   10097 non-null  float64
 8   Wind Spd (km/h)      10219 non-null  float64
 9   Wind Spd Flag        0 non-null      float64
 10  Stn Press (kPa)      10219 non-null  float64
 11  Hmdx                 1153 non-null   float64
 12  Wind Chill           1331 non-null   float64
 13  Load [MW]            10234 non-null  float64
dtypes: float64(10), int64(4)
memory usage: 1.1 MB


In [52]:
data_df.interpolate(method ='linear', limit_direction ='forward')

,Year,Month,Day,Time (LST),Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Wind Spd Flag,Stn Press (kPa),Hmdx,Wind Chill,Load [MW]
0,2021,1,1,10,-1.5,-6.6,68.0,31.0,17.0,NaN,102.85,NaN,-7.000000,1439.43
1,2021,1,1,11,-1.1,-7.9,60.0,31.0,18.0,NaN,102.86,NaN,-6.000000,1457.03
2,2021,1,1,12,-0.4,-7.8,57.0,32.0,18.0,NaN,102.83,NaN,-5.000000,1457.94
3,2021,1,1,13,0.4,-8.3,52.0,32.0,11.0,NaN,102.74,NaN,-4.714286,1444.32
4,2021,1,1,14,1.6,-8.7,46.0,32.0,10.0,NaN,102.74,NaN,-4.428571,1403.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10229,2020,12,31,19,3.3,0.4,81.0,27.0,15.0,NaN,101.49,28.0,-2.000000,1521.92
10230,2020,12,31,20,2.9,-0.3,79.0,27.0,16.0,NaN,101.58,28.0,-2.000000,1463.28
10231,2020,12,31,21,2.3,-0.6,81.0,27.0,12.0,NaN,101.65,28.0,-2.000000,1404.85
10232,2020,12,31,22,2.2,-0.8,81.0,27.0,13.0,NaN,101.70,28.0,-2.000000,1359.48


In [53]:
data_df = data_df.reset_index()
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [61]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)

In [62]:
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

In [63]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 485.391131


In [64]:
print(y_test)

6612    1208.48
9563    1317.79
3438    1177.45
3544    1107.78
4968    1344.07
         ...   
9455    1164.28
3002    1167.71
7504    1149.81
5299    1677.11
2903    1279.27
Name: Load [MW], Length: 2047, dtype: float64


In [65]:
print(preds)

[859.5916 825.7384 786.3335 ... 792.0638 986.9715 789.4423]


In [66]:
print ((y_test - preds).mean())

461.6487832279334
